<div style="display: flex; align-items: center; justify-content: space-between; background-color: #1e355f; padding: 30px;">
    <div>
        <h1 style="color: white; text-align: center; font-weight: bold;">Datos No Estructurados: NLP</h1>
        <h2 style="color: white; text-align: center;">Fake News Detection</h2>
    </div>

</div>


<div style="padding: 5px;">
    <h3 style="color: #1e355f; font-weight: bold;">Realizado por:</h3>
</div>

- Álvaro Ezquerro Pérez
- María Calvo de Mora Román
- Celia Quiles Alemañ


In [29]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

<div style="display: flex; align-items: center; justify-content: space-between; background-color: #1e355f; padding: 10px;">
    <div>
        <h1 style="color: white; text-align: center; font-weight: bold;">Preprocesado de datos para NLP</h1>
    </div>
</div>


El objetivo de este notebook es explicar en profundidad el preprocesado que realizamos posteriormente en todos los notebooks siguientes.

### ¿Por qué es necesario el preprocesado para NLP?

El preprocesamiento de datos es esencial en el procesamiento del lenguaje natural (NLP) por varias razones clave: [1]

1. **Normalización del texto**: El texto en lenguaje natural puede contener una variedad de variaciones y formas, como mayúsculas y minúsculas, abreviaturas, errores tipográficos, etc. Normalizar el texto asegura que esté en un formato consistente para un análisis efectivo.

2. **Eliminación de ruido**: El texto puede contener información irrelevante o ruido, como etiquetas HTML, símbolos de puntuación, números, caracteres especiales, etc. Eliminar este ruido puede mejorar la calidad de los datos y hacer que el análisis sea más preciso.

3. **Tokenización**: La tokenización implica dividir el texto en unidades más pequeñas, como palabras o subpalabras (tokens). Este paso es fundamental para muchos modelos de NLP, ya que permite al modelo comprender la estructura y el significado del texto.

4. **Eliminación de stop words**: Las palabras comunes que no aportan mucho significado al análisis, como "el", "la", "a", "de", etc., a menudo se eliminan durante el preprocesamiento para reducir el ruido y mejorar la eficiencia del análisis.

5. **Stemming y lematización**: Estos procesos reducen las palabras a sus formas básicas o raíces, lo que ayuda a consolidar el vocabulario y a capturar el significado esencial de las palabras. Por ejemplo, "corriendo", "corrió" y "corre" pueden reducirse todas a "correr".

6. **Codificación de texto**: Convertir el texto en un formato numérico que los algoritmos de aprendizaje automático puedan entender es fundamental. Esto puede implicar la codificación de palabras como números enteros o vectores, utilizando técnicas como Bag of Words (BoW) o TF-IDF (Term Frequency-Inverse Document Frequency).

Es decir, el preprocesamiento de datos en NLP es necesario para limpiar y estructurar los datos de texto de manera que los modelos de aprendizaje automático puedan entenderlos y extraer información significativa. Esto no solo mejora la precisión del análisis, sino que también ayuda a reducir el tiempo de procesamiento y mejora la eficiencia del modelo. [1]

-----

Como veremos en todos los siguientes notebooks, el preprocesado lo realizaremos de un golpe mediante una función que combina todas las transformaciones que hemos considerado necesarias.

Sin embargo, en este notebook mostraremos con un ejemplo, qué hace esta función paso a paso.

La función que desglosaremos en la siguiente:

In [30]:
ps = PorterStemmer()
def stemming(content):
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    stemmed_content = stemmed_content.lower()
    stemmed_content = stemmed_content.split()
    stemmed_content = [ps.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    stemmed_content = ' '.join(stemmed_content)

    return stemmed_content

Para hacer la explicación, usaré el siguiente titular de una Fake New extraída del dataset que estamos empleando en este trabajo:

`"BREAKING NEWS: Vladimir Putin Retaliates After New Sanctions Are Slapped On Russia".`

Consiste en un titular con mayúsculas, nombres propios y signos de puntuación, entre otros.

In [31]:
texto_ejemplo = "BREAKING NEWS: Vladimir Putin Retaliates After New Sanctions Are Slapped On Russia"

#### 1) Eliminamos caracteres que no son letras

Empezamos eliminando caracteres que no son letras (como números, signos de puntuación, etc.). Así nos aseguramos de que el texto consista solo en palabras relevantes para el análisis, lo que simplifica el procesamiento y mejora la calidad de los datos.

In [32]:
texto_limpio = re.sub('[^a-zA-Z]',' ', texto_ejemplo)

print("Texto sin caracteres especiales:", texto_limpio)

Texto sin caracteres especiales: BREAKING NEWS  Vladimir Putin Retaliates After New Sanctions Are Slapped On Russia


Como se aprecia en la salida, ya no están los dos puntos (`:`) que sí había antes.

#### 2) Convertimos todo el texto a minúsculas

En segundo lugar, transformamos las mayúsculas.

Al convertir todas las letras del texto a minúsculas, evitamos problemas de sensibilidad a mayúsculas y minúsculas. Esto garantiza que las palabras con la misma raíz, pero escritas de manera diferente, se traten de la misma manera durante el análisis.

In [33]:
texto_limpio_minusculas = texto_limpio.lower()

print("Texto en minúsculas:", texto_limpio_minusculas)

Texto en minúsculas: breaking news  vladimir putin retaliates after new sanctions are slapped on russia


#### 3) Dividimos en palabras

Este tercer paso de dividir el texto en un vector de múltiples palabras realmente solo tiene el objetivo de habilitar la posibilidad de realizar las transformaciones de los pasos 4 y 5, las cuales se realizan a nivel de palabra.

No obstante, tras esas dos transfromaciones, volveremos a unir todas las palabras en el paso 6.

In [34]:
palabras = texto_limpio_minusculas.split()

print("Palabras:", palabras)
print("Número de palabras:", len(palabras))

Palabras: ['breaking', 'news', 'vladimir', 'putin', 'retaliates', 'after', 'new', 'sanctions', 'are', 'slapped', 'on', 'russia']
Número de palabras: 12


#### 4) Eliminamos stopwords

Las stopwords son palabras comunes que no aportan mucho significado al análisis, como "el", "la", "a", "de", etc. Al eliminar estas palabras, reducimos el ruido en los datos y nos centramos en las palabras más informativas para el análisis.

Como los titulares analizados son en inglés, cogemos una lista de stopwords de lengua inglesa.

In [35]:
palabras_sin_stopwords = [word for word in palabras if word not in stopwords.words('english')]

print("Palabras sin stopwords:", palabras_sin_stopwords)
print("Número de palabras:", len(palabras_sin_stopwords))

Palabras sin stopwords: ['breaking', 'news', 'vladimir', 'putin', 'retaliates', 'new', 'sanctions', 'slapped', 'russia']
Número de palabras: 9


Como se aprecia, hemos pasado de tener 12 palabras en el titular, a tener 9. Se han eliminado 'after', 'are' y 'on'.

#### 5) Aplicamos stemming

El stemming es el proceso de reducir las palabras a su forma base o raíz. Esto ayuda a consolidar el vocabulario y a capturar el significado esencial de las palabras, lo que facilita el análisis al tratar las variantes de una palabra como una sola entidad.

In [36]:
palabras_stemmed = [ps.stem(word) for word in palabras_sin_stopwords]

print("Palabras stemmed:", palabras_stemmed)

Palabras stemmed: ['break', 'news', 'vladimir', 'putin', 'retali', 'new', 'sanction', 'slap', 'russia']


Por ejemplo, 'breaking' ha pasado a ser 'break', 'sanctions' ahora es 'sanction', o 'slapped' es ahora 'slap'.

#### 6) Unimos las palabras

Después de aplicar stemming y eliminar stopwords, unimos las palabras nuevamente en un único string. Esto nos da el texto preprocesado y listo para la siguiente etapa del análisis, la vectorización.

In [37]:
texto_final = ' '.join(palabras_stemmed)

print("Texto final:", texto_final)

Texto final: break news vladimir putin retali new sanction slap russia


#### 7) Vectorizamos

La vectorización convierte el texto preprocesado en una representación numérica que los algoritmos de aprendizaje automático pueden entender. En el caso de TF-IDF vectorization, se asignan valores numéricos a cada palabra basados en su frecuencia en el documento y en el corpus, lo que permite al modelo entender la importancia relativa de cada palabra en el texto.

In [38]:
# Inicializamos el vectorizador TF-IDF
vectorizador = TfidfVectorizer()

# aplicamos vectorizador
texto_vectorizado = vectorizador.fit_transform([texto_final])

print("Texto vectorizado:", texto_vectorizado)

Texto vectorizado:   (0, 5)	0.3333333333333333
  (0, 7)	0.3333333333333333
  (0, 6)	0.3333333333333333
  (0, 1)	0.3333333333333333
  (0, 4)	0.3333333333333333
  (0, 3)	0.3333333333333333
  (0, 8)	0.3333333333333333
  (0, 2)	0.3333333333333333
  (0, 0)	0.3333333333333333


Ahora ya sí, tenemos una salida numérica, entendible por el ordenador.

------
Con esto, finalizamos la explicación del preprocesado, que será ejecutado en los siguientes notebooks.

---
---

<div style="padding: 5px;">
    <h2 style="color: #1e355f; font-weight: bold;">Referencias:</h2>
</div>


[1] ¿Qué es el procesamiento de lenguaje natural? - Explicación del procesamiento de lenguaje natural - AWS. (s. f.). Amazon Web Services, Inc. https://aws.amazon.com/es/what-is/nlp/

